In [1]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer, TransformerEncoder
from torch.utils.data import dataset

In [2]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int, nlayers: int, dropout: float=0.5):
        """
        Args:
            ntoken (int): size of the vocabulary
            d_model (int): features size / embedding dimension
            nhead (int): number of Attention head
            d_hid (int): number of dimension of feedfoward network
            nlayers (int): number of encoder layer
            dropout (float, optional): dropout rate. Defaults to 0.5.
        """
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.embedding = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.linear = nn.Linear(d_model, ntoken)

        self.init_weights()
        
    def init_weights(self) -> None:
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor = None) -> Tensor:
        """
        Arguments:
            src: Tensor, shape ``[seq_len, batch_size]``
            src_mask: Tensor, shape ``[seq_len, seq_len]``

        Returns:
            output Tensor of shape ``[seq_len, batch_size, ntoken]``
        """
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.linear(output)
        return output

In [3]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [4]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [5]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# ``train_iter`` was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into ``bsz`` separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Arguments:
        data: Tensor, shape ``[N]``
        bsz: int, batch size

    Returns:
        Tensor of shape ``[N // bsz, bsz]``
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape ``[seq_len, batch_size]``
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

C:\Users\hungd\AppData\Local\Temp\ipykernel_25708\3517878607.py:12: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]


In [11]:
[torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in train_iter]

[tensor([], dtype=torch.int64),
 tensor([   9, 3849, 3869,  881,    9]),
 tensor([], dtype=torch.int64),
 tensor([20000,    83,  3849,    88,     0,  3869,    21,   780, 28780,     2,
          6182,     3,  3849,     4,     1,  5023,    88,    20,     2,  1837,
          1018,     7,    14,  3849,  3869,   881,   629,   976,     2,    23,
             8,  5790,   299,    12,   575,   232,    67,   452,    19, 13722,
             5,   757,     3,  2500,    17,     1,  1767,  5637,     3,   155,
             6,   246,   354,     6,   976,     2,    24,    23,     1,   237,
            67,     6,     1,  3849,    93,     3,     0,     1,   156,  4419,
             4,  5790,     5,   729,    12,    58,  2096,    14,    43,  7075,
             2,     1,   333,  1085,  3218,     7,     1,    37,    67,     5,
          1694,     1, 11219,     2,     8, 19698,   313,  1063,  2082,     1,
          1702,     4, 19009,    56,     1,    95, 25357,   107,    52,  1938,
          1644,   288,   5

In [6]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape ``[full_seq_len, batch_size]``
        i: int

    Returns:
        tuple (data, target), where data has shape ``[seq_len, batch_size]`` and
        target has shape ``[seq_len * batch_size]``
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [7]:
import copy
import time

ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 2  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 2  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # only on last batch
            src_mask = src_mask[:seq_len, :seq_len]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [8]:
best_val_loss = float('inf')
epochs = 3

with TemporaryDirectory() as tempdir:
    best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model)
        val_loss = evaluate(model, val_data)
        val_ppl = math.exp(val_loss)
        elapsed = time.time() - epoch_start_time
        print('-' * 89)
        print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
            f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_params_path)

        scheduler.step()
    model.load_state_dict(torch.load(best_model_params_path)) # load best model states

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 39.19 | loss  8.09 | ppl  3268.47
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 35.06 | loss  6.87 | ppl   965.44
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 34.60 | loss  6.44 | ppl   626.99
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 34.74 | loss  6.30 | ppl   545.66
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 34.63 | loss  6.19 | ppl   487.58
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 34.18 | loss  6.16 | ppl   472.09
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 34.75 | loss  6.11 | ppl   449.25
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 35.24 | loss  6.11 | ppl   451.06
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 35.04 | loss  6.02 | ppl   411.01
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 34.85 | loss  6.02 | ppl   412.37
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 35.00 | loss  5.90 | ppl   364.16
| epoch   1 |  2400/ 

In [9]:
test_loss = evaluate(model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  5.50 | test ppl   243.95


In [23]:
data, targets = get_batch(train_data, 2)
mask = generate_square_subsequent_mask(bptt).to(device)
output = model(data, mask)

In [25]:
output.size()

torch.Size([35, 20, 28782])

In [46]:
torch.multinomial(
    torch.softmax(output, dim=-1, dtype=torch.float)[:,-1,:],
    num_samples=1
)


tensor([[   1],
        [  29],
        [   2],
        [  50],
        [  12],
        [  35],
        [   8],
        [   6],
        [2916],
        [   5],
        [8304],
        [  31],
        [1250],
        [   6],
        [   0],
        [ 139],
        [ 102],
        [  53],
        [1302],
        [ 367],
        [   7],
        [4766],
        [ 359],
        [5983],
        [   2],
        [ 416],
        [  46],
        [  20],
        [1926],
        [   6],
        [   7],
        [ 403],
        [2292],
        [3025],
        [  24]], device='cuda:0')

: 

In [33]:
train_data[1:3]

tensor([[ 3849,    12,   300,  6302,  3989,  1930, 10559,   451,     4,     7,
             2,  1511, 10115,   942,  2439,   572,     1,    47,    30,  1990],
        [ 3869,   315,    19,    29,   939,     2,    10,  2139,  4916, 16615,
           235,     3,    13,     7,    24,    17, 13737,    97,  7720,     4]],
       device='cuda:0')

In [34]:
train_data[1:3].reshape(-1)

tensor([ 3849,    12,   300,  6302,  3989,  1930, 10559,   451,     4,     7,
            2,  1511, 10115,   942,  2439,   572,     1,    47,    30,  1990,
         3869,   315,    19,    29,   939,     2,    10,  2139,  4916, 16615,
          235,     3,    13,     7,    24,    17, 13737,    97,  7720,     4],
       device='cuda:0')